In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('dataframe').getOrCreate()
spark

In [6]:
df_pyspark = spark.read.csv('people4.csv', header = True, inferSchema=True)
df_pyspark.show()

+---------+---+----------+------+
|     name|age|experience|salary|
+---------+---+----------+------+
|      Kin| 19|         6| 30000|
|     Sawa| 17|         5| 23000|
|     Toru| 22|         9| 40000|
|    Souta| 13|         3| 13000|
|    Kanna|  9|         2|  5000|
|Kobayahsi| 34|        30|400000|
+---------+---+----------+------+



In [7]:
df_pyspark.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- experience: integer (nullable = true)
 |-- salary: integer (nullable = true)



In [8]:
training = df_pyspark

In [11]:
training.columns

['name', 'age', 'experience', 'salary']

In [15]:
#group training features together using vector assembler
train_features = ['age', 'experience']

In [16]:
from pyspark.ml.feature import VectorAssembler
feature_assembler = VectorAssembler(inputCols= train_features, outputCol= "Independent Features")

In [17]:
output = feature_assembler.transform(training)

In [23]:
output.show()

+---------+---+----------+------+--------------------+
|     name|age|experience|salary|Independent Features|
+---------+---+----------+------+--------------------+
|      Kin| 19|         6| 30000|          [19.0,6.0]|
|     Sawa| 17|         5| 23000|          [17.0,5.0]|
|     Toru| 22|         9| 40000|          [22.0,9.0]|
|    Souta| 13|         3| 13000|          [13.0,3.0]|
|    Kanna|  9|         2|  5000|           [9.0,2.0]|
|Kobayahsi| 34|        30|400000|         [34.0,30.0]|
+---------+---+----------+------+--------------------+



In [24]:
output.columns

['name', 'age', 'experience', 'salary', 'Independent Features']

In [25]:
finalized_data = output.select("Independent Features", "salary")
finalized_data.show()

+--------------------+------+
|Independent Features|salary|
+--------------------+------+
|          [19.0,6.0]| 30000|
|          [17.0,5.0]| 23000|
|          [22.0,9.0]| 40000|
|          [13.0,3.0]| 13000|
|           [9.0,2.0]|  5000|
|         [34.0,30.0]|400000|
+--------------------+------+



In [26]:
from pyspark.ml.regression import LinearRegression
train_data, test_date = finalized_data.randomSplit([0.7, 0.3])
regressor = LinearRegression(featuresCol='Independent Features', labelCol= 'salary')
regressor = regressor.fit(train_data)

In [32]:
print(regressor.coefficients)
print(regressor.intercept)

[-7636.363636365045,20272.72727272832]
51454.54545456228


In [34]:
pred_res = regressor.evaluate(test_date)

In [38]:
pred_res.predictions.show()

+--------------------+------+------------------+
|Independent Features|salary|        prediction|
+--------------------+------+------------------+
|           [9.0,2.0]|  5000|23272.727272733508|
|          [19.0,6.0]| 30000|27999.999999996333|
|          [22.0,9.0]| 40000| 65909.09090908615|
+--------------------+------+------------------+



In [39]:
pred_res.meanAbsoluteError

15393.93939394111